In [12]:
#Name: PRISMA2GeoTIFF.py
#Description: reads he5 PRISMA files content and converts it to a GeoTIFF.
#All 66 VNIR bands and 173 SWIR bands are converted in one single GeoTIFF file.
#input is a PRISMA he5 file and output is a GeoTIFF with the same name in the same path
#Author: martin rapilly, mrapilly60@uasd.edu.do/martin.rapilly@get.omp.eu

# import libraries
import h5py
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
from itertools import chain
from osgeo import gdal, osr
import os

#### Previous code - not working

In [14]:
#np.set_printoptions(threshold=np.inf)#optional: uncommenting this line will show full arrays when printing on the console. Not recommended as he5 PRISMA files contain many values that can overrun memory

def PRISMA2GeoTIFF (filename):
    #open he5 file and read its content
    f = h5py.File(filename,'r')
    def print_name(name, obj):
        if isinstance(obj, h5py.Dataset):
            print ('Dataset:', name)
        elif isinstance(obj, h5py.Group):
            print ('Group:', name)
    with h5py.File(filename, 'r')  as h5f: # file will be closed when we exit from WITH scope
        h5f.visititems(print_name)

    
        #read SWIR and VNIR cube contents
        SWIRcube = h5f['HDFEOS/SWATHS/PRS_L2D_HCO/Data Fields/SWIR_Cube'][()]#[()] is to get the value. Can be replaced with .value
        VNIRcube = h5f['HDFEOS/SWATHS/PRS_L2D_HCO/Data Fields/VNIR_Cube'][()]
    
        #read latitude and longitude contents
        lat = h5f['HDFEOS/SWATHS/PRS_L2D_HCO/Geolocation Fields/Latitude'][()]
        lon = h5f['HDFEOS/SWATHS/PRS_L2D_HCO/Geolocation Fields/Longitude'][()]

        #checks SWIR, VNIR and latitude/longitude array shapes
        print ("SWIRcube.shape",SWIRcube.shape)
        print ("VNIRcube.shape",VNIRcube.shape)
        print ("lat.shape",lat.shape)        
        
        #create lists from latitude/longitude values
        lonIter=list(chain.from_iterable(lon))
        latIter=list(chain.from_iterable(lat))


        
        
        #create a list from VNIR and SWIR cube values
        listBand=[]
        for band in range(0,VNIRcube.shape[1]):#VNIRcube.shape[1] gives the number of bands (:66)
            for x in range(0,lat.shape[0]):#lat.shape[0] gives the number of rows
                element=VNIRcube[x][band]
                listBand.append(element)
        for band1 in range(0,SWIRcube.shape[1]):#SWIRcube.shape[1] gives the number of bands (:137)
            for x1 in range(0,lat.shape[0]):#lat.shape[0] gives the number of rows
                element=SWIRcube[x1][band1]
                listBand.append(element)

        #convert list with values to a numpy array      
        data=np.array(listBand,dtype=np.uint16)

        #checks array shape
        print ("data.shape",data.shape)

        #reshape numpy array with the right number of bands, rows and columns
        dataReshaped=data.reshape([VNIRcube.shape[1]+SWIRcube.shape[1], lat.shape[0], lat.shape[1]])
        print ("reshaped data.shape",dataReshaped.shape)

        #get minimum and maximum latitude and longitude
        xmin,ymin,xmax,ymax = [lon.min(),lat.min(),lon.max(),lat.max()]

        #get pixel spatial resolution
        xres = (xmax-xmin)/lat.shape[1]#lat.shape[1] gives the number of cols
        yres = (ymax-ymin)/lat.shape[0]#lat.shape[0] gives the number of rows

        #define coordinates
        geotransform=(xmin,xres,0,ymax,0, -yres)#zeros (third and fifth parameters) are for rotation

        #define GeoTIFF structure and output filename
        output_raster = gdal.GetDriverByName('GTiff').Create(filename [:-3]+"tif",lat.shape[1], lat.shape[0], VNIRcube.shape[1]+SWIRcube.shape[1] ,gdal.GDT_Float32)  # Open the file
        
        #loop over all bands and write it to the GeoTIFF
        for b in range(1,VNIRcube.shape[1]+SWIRcube.shape[1]):
            print("converting band",b)
            outband = output_raster.GetRasterBand(b) 
            outband.WriteArray(dataReshaped[b,:,:])
        #specify coordinates to WGS84
        output_raster.SetGeoTransform(geotransform)  
        srs = osr.SpatialReference()                 
        srs.ImportFromEPSG(4326)                                                               
        output_raster.SetProjection(srs.ExportToWkt())

        #clean memory     
        output_raster.FlushCache()
        print("Conversion from he5 PRISMA file to GeoTIFF complete.")

In [15]:
###Put one or many files in a folder. Modify the folder path and run this part of the code:

In [ ]:
#enter folder path with he5 PRISMA files in it
folderPath= "C:\\Users\\bd167\\OneDrive - University of Leicester\\Documents\\Data\\Satellite_data\\PRISMA"
listImages=[]
for file in os.listdir(folderPath):
      listImages.append(os.path.join(folderPath, file))
print ("he5 image list: ", listImages)

#apply function PRISMA2GeoTIFF
for filename in listImages:
    print("Processing image", filename)
    PRISMA2GeoTIFF(filename)
print ("All files processed.")

#### Error above? check file paths and redo

In [17]:
# def print_structure(filename):
#     with h5py.File(filename, 'r') as f:
#         def print_name(name, obj):
#             print(name)
#         f.visititems(print_name)

# # Test this on one of your files
# filename = "C:\\Users\\bd167\\OneDrive - University of Leicester\\Documents\\Data\\Satellite_data\\PRISMA\\PRS_L2C_STD_20240829113445_20240829113449_0001.he5"
# print_structure(filename)

In [ ]:
def is_hdf5(filename):
    """Check if a file is a valid HDF5 file."""
    try:
        with h5py.File(filename, 'r'):
            return True
    except OSError:
        return False

def PRISMA2GeoTIFF(filename):
    # Check if the file is a valid HDF5 file
    if not is_hdf5(filename):
        print(f"Skipping non-HDF5 file: {filename}")
        return

    # Proceed with the actual file processing
    print(f"Processing valid HDF5 file: {filename}")

    # Open HDF5 file and read its content
    with h5py.File(filename, 'r') as h5f:
        # Read SWIR and VNIR cube contents
        try:
            SWIRcube = h5f['HDFEOS/SWATHS/PRS_L2C_HCO/Data Fields/SWIR_Cube'][()]  # Correct path for SWIR_Cube
            VNIRcube = h5f['HDFEOS/SWATHS/PRS_L2C_HCO/Data Fields/VNIR_Cube'][()]  # Correct path for VNIR_Cube
        except KeyError as e:
            print(f"Error: {e}. The dataset could not be found in the file {filename}")
            return
        
        # Read latitude and longitude contents
        lat = h5f['HDFEOS/SWATHS/PRS_L2C_HCO/Geolocation Fields/Latitude'][()]
        lon = h5f['HDFEOS/SWATHS/PRS_L2C_HCO/Geolocation Fields/Longitude'][()]

        # Check SWIR, VNIR, and latitude/longitude array shapes
        print("SWIRcube.shape", SWIRcube.shape)
        print("VNIRcube.shape", VNIRcube.shape)
        print("lat.shape", lat.shape)        

        # Create lists from latitude/longitude values
        lonIter = list(chain.from_iterable(lon))
        latIter = list(chain.from_iterable(lat))

        # Create a list from VNIR and SWIR cube values
        listBand = []
        for band in range(0, VNIRcube.shape[1]):  # VNIRcube.shape[1] gives the number of bands (:66)
            for x in range(0, lat.shape[0]):  # lat.shape[0] gives the number of rows
                element = VNIRcube[x][band]
                listBand.append(element)
        for band1 in range(0, SWIRcube.shape[1]):  # SWIRcube.shape[1] gives the number of bands (:137)
            for x1 in range(0, lat.shape[0]):  # lat.shape[0] gives the number of rows
                element = SWIRcube[x1][band1]
                listBand.append(element)

        # Convert list with values to a numpy array      
        data = np.array(listBand, dtype=np.uint16)

        # Check array shape
        print("data.shape", data.shape)

        # Reshape numpy array with the right number of bands, rows, and columns
        dataReshaped = data.reshape([VNIRcube.shape[1] + SWIRcube.shape[1], lat.shape[0], lat.shape[1]])
        print("reshaped data.shape", dataReshaped.shape)

        # Get minimum and maximum latitude and longitude
        xmin, ymin, xmax, ymax = [lon.min(), lat.min(), lon.max(), lat.max()]

        # Get pixel spatial resolution
        xres = (xmax - xmin) / lat.shape[1]  # lat.shape[1] gives the number of cols
        yres = (ymax - ymin) / lat.shape[0]  # lat.shape[0] gives the number of rows

        # Define coordinates
        geotransform = (xmin, xres, 0, ymax, 0, -yres)  # zeros (third and fifth parameters) are for rotation

        # Define GeoTIFF structure and output filename
        output_raster = gdal.GetDriverByName('GTiff').Create(filename[:-3] + "tif", lat.shape[1], lat.shape[0], VNIRcube.shape[1] + SWIRcube.shape[1], gdal.GDT_Float32)

        # Loop over all bands and write them to the GeoTIFF
        for b in range(1, VNIRcube.shape[1] + SWIRcube.shape[1]):
            print("converting band", b)
            outband = output_raster.GetRasterBand(b)
            outband.WriteArray(dataReshaped[b, :, :])

        # Specify coordinates to WGS84
        output_raster.SetGeoTransform(geotransform)
        srs = osr.SpatialReference()
        srs.ImportFromEPSG(4326)
        output_raster.SetProjection(srs.ExportToWkt())

        # Clean memory     
        output_raster.FlushCache()
        print("Conversion from he5 PRISMA file to GeoTIFF complete.")

# Get the list of images and process them
folderPath = "C:\\Users\\bd167\\OneDrive - University of Leicester\\Documents\\Data\\Satellite_data\\PRISMA"
listImages = [os.path.join(folderPath, file) for file in os.listdir(folderPath) if file.endswith('.he5')]

# Apply function PRISMA2GeoTIFF to valid HDF5 files
for filename in listImages:
    print("Processing image:", filename)
    PRISMA2GeoTIFF(filename)

print("All files processed.")


#### Further mod - rotation by 90degrees

In [ ]:
def is_hdf5(filename):
    """Check if a file is a valid HDF5 file."""
    try:
        with h5py.File(filename, 'r'):
            return True
    except OSError:
        return False

def PRISMA2GeoTIFF(filename):
    if not is_hdf5(filename):
        print(f"Skipping non-HDF5 file: {filename}")
        return

    print(f"Processing valid HDF5 file: {filename}")

    with h5py.File(filename, 'r') as h5f:
        try:
            SWIRcube = h5f['HDFEOS/SWATHS/PRS_L2C_HCO/Data Fields/SWIR_Cube'][()]  
            VNIRcube = h5f['HDFEOS/SWATHS/PRS_L2C_HCO/Data Fields/VNIR_Cube'][()]  
        except KeyError as e:
            print(f"Error: {e}. The dataset could not be found in the file {filename}")
            return

        lat = h5f['HDFEOS/SWATHS/PRS_L2C_HCO/Geolocation Fields/Latitude'][()]
        lon = h5f['HDFEOS/SWATHS/PRS_L2C_HCO/Geolocation Fields/Longitude'][()]

        listBand = []
        for band in range(0, VNIRcube.shape[1]):
            for x in range(0, lat.shape[0]):
                element = VNIRcube[x][band]
                listBand.append(element)
        for band1 in range(0, SWIRcube.shape[1]):
            for x1 in range(0, lat.shape[0]):
                element = SWIRcube[x1][band1]
                listBand.append(element)

        data = np.array(listBand, dtype=np.uint16)

        dataReshaped = data.reshape([VNIRcube.shape[1] + SWIRcube.shape[1], lat.shape[0], lat.shape[1]])
        print("reshaped data.shape", dataReshaped.shape)

        # Rotate data if necessary
        dataReshaped = np.rot90(dataReshaped, k=3, axes=(1, 2))  # Rotate to correct orientation

        xmin, ymin, xmax, ymax = [lon.min(), lat.min(), lon.max(), lat.max()]

        xres = (xmax - xmin) / lat.shape[1]
        yres = (ymax - ymin) / lat.shape[0]

        geotransform = (xmin, xres, 0, ymax, 0, -yres)

        output_raster = gdal.GetDriverByName('GTiff').Create(filename[:-3] + "tif", lat.shape[1], lat.shape[0], VNIRcube.shape[1] + SWIRcube.shape[1], gdal.GDT_Float32)

        for b in range(1, VNIRcube.shape[1] + SWIRcube.shape[1]):
            outband = output_raster.GetRasterBand(b)
            outband.WriteArray(dataReshaped[b, :, :])

        output_raster.SetGeoTransform(geotransform)
        srs = osr.SpatialReference()
        srs.ImportFromEPSG(4326)
        output_raster.SetProjection(srs.ExportToWkt())

        output_raster.FlushCache()
        print("Conversion from he5 PRISMA file to GeoTIFF complete.")


# Get the list of images and process them
folderPath = "C:\\Users\\bd167\\OneDrive - University of Leicester\\Documents\\Data\\Satellite_data\\PRISMA"
listImages = [os.path.join(folderPath, file) for file in os.listdir(folderPath) if file.endswith('.he5')]

# Apply function PRISMA2GeoTIFF to valid HDF5 files
for filename in listImages:
    print("Processing image:", filename)
    PRISMA2GeoTIFF(filename)

print("All files processed.")
